In [1]:
attacks = ['bim', 'fgsm', 'pgd', 'df', 'cw']
sizes = [32, 64, 128, 256, 512, 1024]

files = []
for x in sizes:
    for y in attacks:
        files = files + [f"./generated_images-test/{x}x{x}/metadata_{y}.csv"]

print (files)

['./generated_images-test/32x32/metadata_bim.csv', './generated_images-test/32x32/metadata_fgsm.csv', './generated_images-test/32x32/metadata_pgd.csv', './generated_images-test/32x32/metadata_df.csv', './generated_images-test/32x32/metadata_cw.csv', './generated_images-test/64x64/metadata_bim.csv', './generated_images-test/64x64/metadata_fgsm.csv', './generated_images-test/64x64/metadata_pgd.csv', './generated_images-test/64x64/metadata_df.csv', './generated_images-test/64x64/metadata_cw.csv', './generated_images-test/128x128/metadata_bim.csv', './generated_images-test/128x128/metadata_fgsm.csv', './generated_images-test/128x128/metadata_pgd.csv', './generated_images-test/128x128/metadata_df.csv', './generated_images-test/128x128/metadata_cw.csv', './generated_images-test/256x256/metadata_bim.csv', './generated_images-test/256x256/metadata_fgsm.csv', './generated_images-test/256x256/metadata_pgd.csv', './generated_images-test/256x256/metadata_df.csv', './generated_images-test/256x256/m

In [2]:
import math
from pathlib import Path
import pandas as pd

OUT_DIR = Path('./Evaluate')
OUT_DIR.mkdir(exist_ok=True)

paths = files

def parse_success_value(v):
    if pd.isna(v):
        return None
    if isinstance(v, bool):
        return v
    if isinstance(v, (int, float)) and not math.isnan(v):
        try:
            return bool(int(v))
        except Exception:
            return None
    s = str(v).strip().lower()
    if s in ('true','1','yes','y','t','success','succeeded'):
        return True
    if s in ('false','0','no','n','f','none',''):
        return False
    try:
        iv = int(float(s))
        return bool(iv)
    except Exception:
        return None

per_file_success = []
missing_files = []

for p in paths:
    ppath = Path(p)
    if not ppath.exists():
        missing_files.append(str(ppath))
        continue
    
    try:
        df = pd.read_csv(ppath)
    except Exception:
        try:
            df = pd.read_csv(ppath, engine='python', encoding='utf-8', error_bad_lines=False)
        except Exception:
            missing_files.append(str(ppath))
            continue
    
    success_col = 'success'
   
    if success_col:
        success_mask = [parse_success_value(v) is True for v in df[success_col]]
    elif 'misclassified_class' in df.columns:
        success_mask = [False if pd.isna(v) or str(v).strip().lower() in ('','none') else True for v in df['misclassified_class']]
    else:
        success_mask = [False] * len(df)
    
    success_df = df.loc[success_mask, ['rel_path', 'true_idx', 'pred_idx', 'true_class', 'pred_class']].copy()
    per_file_success.append(success_df)

if per_file_success:
    common_relpaths = set(per_file_success[0]['rel_path'])
    for sdf in per_file_success[1:]:
        common_relpaths &= set(sdf['rel_path'])
else:
    common_relpaths = set()

if common_relpaths:
    common_df = per_file_success[0][per_file_success[0]['rel_path'].isin(common_relpaths)].copy()
else:
    common_df = pd.DataFrame(columns=['rel_path', 'true_idx', 'pred_idx', 'true_class', 'pred_class'])

out_path = OUT_DIR / 'common_success_details.csv'
common_df.to_csv(out_path, index=False)

print("Missing/unreadable files:", len(missing_files))
print("Common successful rel_paths:", len(common_relpaths))
print("Saved:", out_path)
display(common_df.head(20))


Missing/unreadable files: 3
Common successful rel_paths: 649
Saved: Evaluate\common_success_details.csv


,rel_path,true_idx,pred_idx,true_class,pred_class
0,/fish/212761573.jpg,2,2,fish,fish
9,/frog/193668642.jpg,3,3,frog,frog
15,/monkey/101701289.jpg,4,4,monkey,monkey
17,/monkey/117620887.jpg,4,4,monkey,monkey
23,/cat/298964548.jpg,1,1,cat,cat
24,/bird/294084465.jpg,0,0,bird,bird
31,/frog/216649025.jpg,3,3,frog,frog
35,/frog/129461242.jpg,3,3,frog,frog
36,/bird/206351823.jpg,0,0,bird,bird
41,/moth/288076622.jpg,5,5,moth,moth
